In [ ]:
!pip install transformers
!pip install datasets
!pip install trl

In [ ]:
!pip install trl

In [75]:
ls  ckpts5/  

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
ppo_1/   ppo_5/      ppo_5.128/  ppo_5.158/  ppo_5.187/  ppo_5.46/  ppo_5.75/
ppo_10/  ppo_5.1/    ppo_5.129/  ppo_5.159/  ppo_5.188/  ppo_5.47/  ppo_5.77/
ppo_11/  ppo_5.10/   ppo_5.13/   ppo_5.16/   ppo_5.189/  ppo_5.48/  ppo_5.78/
ppo_12/  ppo_5.100/  ppo_5.130/  ppo_5.160/  ppo_5.19/   ppo_5.49/  ppo_5.79/
ppo_13/  ppo_5.101/  ppo_5.131/  ppo_5.161/  ppo_5.2/    ppo_5.5/   ppo_5.8/
ppo_14/  ppo_5.102/  ppo_5.132/  ppo_5.162/  ppo_5.20/   ppo_5.50/  ppo_5.80/
ppo_15/  ppo_5.103/  ppo_5.133/  ppo_5.163/  ppo_5.21/   ppo_5.51/  ppo_5.81/
ppo_16/  ppo_5.104/  ppo_5.134/  ppo_5.164/  ppo_5.22/   ppo_5.52/  ppo_5.82/
ppo_17/  ppo_5.105/  ppo_5.135/  ppo_5.165/  ppo_5.23

In [76]:
import torch

import time
import os
from tqdm import tqdm
import numpy as np
import pandas as pd


from datasets import load_dataset

from transformers import AutoTokenizer, pipeline
import torch
from tqdm import tqdm
from transformers import pipeline, AutoTokenizer
from datasets import load_dataset
from trl import  AutoModelForCausalLMWithValueHead

In [77]:
ls ckpts5

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
ppo_1/   ppo_5/      ppo_5.128/  ppo_5.158/  ppo_5.187/  ppo_5.46/  ppo_5.75/
ppo_10/  ppo_5.1/    ppo_5.129/  ppo_5.159/  ppo_5.188/  ppo_5.47/  ppo_5.77/
ppo_11/  ppo_5.10/   ppo_5.13/   ppo_5.16/   ppo_5.189/  ppo_5.48/  ppo_5.78/
ppo_12/  ppo_5.100/  ppo_5.130/  ppo_5.160/  ppo_5.19/   ppo_5.49/  ppo_5.79/
ppo_13/  ppo_5.101/  ppo_5.131/  ppo_5.161/  ppo_5.2/    ppo_5.5/   ppo_5.8/
ppo_14/  ppo_5.102/  ppo_5.132/  ppo_5.162/  ppo_5.20/   ppo_5.50/  ppo_5.80/
ppo_15/  ppo_5.103/  ppo_5.133/  ppo_5.163/  ppo_5.21/   ppo_5.51/  ppo_5.81/
ppo_16/  ppo_5.104/  ppo_5.134/  ppo_5.164/  ppo_5.22/   ppo_5.52/  ppo_5.82/
ppo_17/  ppo_5.105/  ppo_5.135/  ppo_5.165/  ppo_5.23

In [103]:

from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('gpt2',padding_side='left')
tokenizer.pad_token = tokenizer.eos_token
gpt2_model = AutoModelForCausalLMWithValueHead.from_pretrained('ckpts5/ppo_5.186/')

Some weights of the model checkpoint at ckpts5/ppo_5.186/ were not used when initializing GPT2LMHeadModel: ['v_head.summary.bias', 'v_head.summary.weight']
- This IS expected if you are initializing GPT2LMHeadModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing GPT2LMHeadModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [104]:
device = 'cuda'

In [105]:
gpt2_model = gpt2_model.to(device)

In [106]:
gen_kwargs = {
    "min_length":20,
    "max_new_tokens":100,
    "top_k": 10,
    "top_p": 0.9,
    "do_sample": True,
    "no_repeat_ngram_size":2,
    "pad_token_id": tokenizer.eos_token_id,
    'use_cache':True
}

In [107]:
def get_answer(q, model:AutoModelForCausalLMWithValueHead, tokenizer:AutoTokenizer, ):
    gen_kwargs = {
    "min_length":20,
    "max_new_tokens":100,
    "top_k": 10,
    "top_p": 0.9,
    "do_sample": True,
    "no_repeat_ngram_size":2,
    "pad_token_id": tokenizer.eos_token_id,
    'use_cache':True
}
    enc  = tokenizer.encode(f'Question: {q} ? Answer:', return_tensors='pt').to(device)
    gen = model.generate(enc, **gen_kwargs)
    return tokenizer.decode(gen[0]).split('Answer:')[1]


# Оценка модели

In [108]:
from torch.utils.data import Dataset
from tqdm.contrib import tzip


class SumDataset(Dataset):
    def __init__(self,tokenizer):
      dataset = load_dataset("neulab/conala",'curated')
      qs = dataset['train']['rewritten_intent']
      ans = dataset['train']['snippet']

      self.tokenized = []
      self.answer = []

      for a, q in tzip(ans,qs):

        pr = f'Question: {q}  Answer:'

        enc = self._encode(text=pr, tokenizer=tokenizer)#@, self._encode(text=q, tokenizer=tokenizer), self._encode(text=a, tokenizer=tokenizer)
        self.tokenized += [enc]
        self.answer += [a]

    def __len__(self):
        return len(self.tokenized)

    def __getitem__(self, item):
        return ([self.tokenized[item], self.answer[item]])

    def _encode(self, text, tokenizer):
        
        encoded_sample = tokenizer.encode(text, padding='max_length', max_length=512, return_tensors='pt')

        return encoded_sample

In [109]:
dataset = load_dataset("neulab/conala",'curated')
qs = dataset['train']['rewritten_intent']

Found cached dataset conala (/home/jovyan/.cache/huggingface/datasets/neulab___conala/curated/1.1.0/8557ab15640c382a1db16dca51ee1845bc77bf7c45015a2c095446a8e20ba5d3)


  0%|          | 0/2 [00:00<?, ?it/s]

In [110]:
qs[:10]

["Concatenate elements of a list 'x' of multiple integers to a single integer",
 'convert a list of integers into a single integer',
 "convert a DateTime string back to a DateTime object of format '%Y-%m-%d %H:%M:%S.%f'",
 'get the average of a list values for each key in dictionary `d`)',
 'zip two lists `[1, 2]` and `[3, 4]` into a list of two tuples containing elements at the same index in each list',
 "prepend string 'hello' to all items in list 'a'",
 'regex for repeating words in a string `s`',
 'normalize a pandas dataframe `df` by row',
 'swap values in a tuple/list inside a list `mylist`',
 'Swap values in a tuple/list in list `mylist`']

In [111]:
from torch.utils.data import DataLoader
data = SumDataset(tokenizer)
test_dataloader = DataLoader(data, batch_size=10)

Found cached dataset conala (/home/jovyan/.cache/huggingface/datasets/neulab___conala/curated/1.1.0/8557ab15640c382a1db16dca51ee1845bc77bf7c45015a2c095446a8e20ba5d3)


  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2379 [00:00<?, ?it/s]

In [112]:
for batch in tqdm(test_dataloader):
    t = batch[0].to(device)
    gen = gpt2_model.generate(t.squeeze(1), **gen_kwargs)
    a = tokenizer.batch_decode(gen)
    break

  0%|          | 0/238 [00:01<?, ?it/s]


In [132]:
qs = dataset['train']['rewritten_intent']
ans = dataset['train']['snippet']

In [135]:
ans[0]

'sum(d * 10 ** i for i, d in enumerate(x[::-1]))'

In [133]:
qs[0]

"Concatenate elements of a list 'x' of multiple integers to a single integer"

In [220]:
t = tokenizer.encode('Question: Write a function to solve x+y Answer:',  max_length=128, return_tensors='pt').to(device)

In [221]:
gen_kwargs = {
    "min_length": 20,
    "max_new_tokens": 100,
    "top_k": 100,
    "top_p": 0.7,
    "do_sample": True,  # Use beam search instead of sampling
    "no_repeat_ngram_size": 2,
    "pad_token_id": tokenizer.eos_token_id,
    "use_cache": True,
    "num_beams": 2,  # Use beam search with 5 beams
    #"repetition_penalty": 1.5,  # Add a penalty for repetition
    #"length_penalty": 0.8  # Add a penalty for long responses
}


In [222]:
g  = gpt2_model.generate(t, **gen_kwargs)

In [223]:
np.array([1, 2, 3, 4]).sum(0)

10

In [224]:
x = [1, 2, 3, 4]

In [225]:
tokenizer.decode(g[0])

'Question: Write a function to solve x+y Answer: x + y\n\nPython dictionary dictionary of tuples containing dictionary values of the form dictionary[key:value]\n<|endoftext|>'

In [118]:
import nltk
def evaluate(model:AutoModelForCausalLMWithValueHead, tokenizer:AutoTokenizer, test_dataloader, device, gen_kwargs):
  scores = []
  ans = []
  ans_gt = []
  for batch in tqdm(test_dataloader):
    t = batch[0].to(device)
    gen = gpt2_model.generate(t.squeeze(1), **gen_kwargs)
    a = tokenizer.batch_decode(gen)
    batch_gen = []
    batch_orig = []
    for i in a:
      batch_gen.append(i.replace('<|endoftext|>', '').split('Answer:')[1].lower().split())
    for i in batch[1]:
      batch_orig.append(i.lower().split())
    ans+=[batch_gen]
    ans_gt = [batch_orig]
    score = nltk.translate.bleu_score.corpus_bleu(hypotheses=batch_gen, list_of_references=batch_orig)
    scores+=[score]
  blue_metric = np.array(scores).mean()*100
  return blue_metric, ans, ans_gt

In [ ]:
blue_metric, ans, ans_gt = evaluate(model=gpt2_model, tokenizer=tokenizer, test_dataloader=test_dataloader,
         device=device, gen_kwargs=gen_kwargs)

 45%|████▍     | 107/238 [09:51<11:13,  5.14s/it]

In [ ]:
blue_metric, ans, ans_gt

In [121]:
blue_metric, , ans_gt

(6.58053872291883e-231,
 [[['x.concat(*x)'],
   ["int('[0-9]+')"],
   ['dt', '=', 'datetime.date(dt)'],
   ['d.items()'],
   ['sorted(map(lambda', 'x:', 'x[0],', 'x))'],
   ["a.append('hello')"],
   ["re.findall('\\\\d+',", 's)'],
   ['df.reset_index()'],
   ['mylist.sort(key=lambda', 'x:', 'x[1])'],
   ['mylist.sort(key=lambda', 'x:', 'x[1])']],
  [['map(str,', 'dict)'],
   ['strs.findall(pattern=str)'],
   ['random.sample(range(3))'],
   ['df.fit(lambda', 'x:', 'x[1])'],
   ["soup.findall('a')"],
   ['date.date()'],
   ["s.encode('hex')"],
   ["dic.setdefault(['test',", '5])'],
   ['sorted(s,', 'key=str)'],
   ['map(str,', 'dict)']],
  [['np.unique(vals,', 'key=lambda', 'x:', 'x[1])'],
   ['map(str,', 'dict)'],
   ["dict([{'two':", '2,', "'one':", '1})'],
   ['dict([k,', 'v])'],
   ['gravity.setdefault(0)'],
   ["re.findall('([0-9]+)([a-z]))"],
   ["re.sub('([0-9]+)([a-z]))"],
   ["re.sub('([0-9]+)([a-z]))"],
   ['dictionary.items()'],
   ['time.sleep(1)']],
  [['urllib.request.urlop

In [131]:
ans_gt

[[['json.dumps(your_data,', 'ensure_ascii=false)'],
  ['values',
   '=',
   'np.array([i',
   'for',
   'i',
   'in',
   'range(100)],',
   'dtype=np.float64)'],
  ['sorted(list_of_dct,',
   'key=lambda',
   'x:',
   'order.index(list(x.values())[0]))'],
  ['return', 's[0].upper()', '+', 's[1:]'],
  ['"""""".join([1,', '2,', '3,', '4])'],
  ['line', '=', "line.decode('utf-8',", "'ignore').encode('utf-8')"],
  ['os.system(command)'],
  ["c.execute('select",
   '*',
   'from',
   'foo',
   'where',
   'bar',
   '=',
   '%s',
   'and',
   'baz',
   '=',
   "%s',",
   '(param1,',
   'param2))'],
  ['dateobj',
   '=',
   'datetime.datetime.strptime(datestr,',
   "'%y-%m-%d').date()"]]]

In [128]:
ans[3]

[['urllib.request.urlopen(url)'],
 ['np.zeros((a,', '-1))'],
 ['dict([k,', 'v])'],
 ['dict([k,', 'v])'],
 ['dict([k,', 'v])'],
 ["re.findall('\\\\d+',", 's)'],
 ['s.setopt(socket.sol_socket,', '(1,', '1))'],
 ['sum(map(lambda', 'x:', 'x[1]))'],
 ["s.decode('hex')"],
 ['sorted_list_of_values.sort(key=lambda', 'x:', 'x[1])']]

IndexError: list index out of range

In [14]:
ans[0]

[['multi(map(int,', "'a')))"],
 ['cube(x,', 'y)'],
 ["datetime.strftime('%y-", '%m,%s', '%h:', "%m',", "'%.date())"],
 ['median(d,', 'key=lambda', 'x:', 'int)', '-', 'gamma(x))'],
 ['map(x,', 'y)', '______________________________'],
 ['cla[0]', '_____'],
 ["snake.rstrip('',", "'',", '1)[0]'],
 ['pd.norm()'],
 ['sadx[i]', '______________________________________'],
 ['polar(a,', 'b)']]

In [57]:
results = [''.join(lst) for lst in ans[0]]

In [ ]:
ans_gt